## `fastplotlib` demo

In [7]:
import numpy as np
import zmq
import fastplotlib as fpl
from wgpu.backends import wgpu_native

### Check if GPU or rendering with CPU via lavapipe is available

In [8]:
if len(wgpu_native.enumerate_adapters()) < 1:
  raise IOError("No WGPU compatible adapter found")

### Setup zmq subscriber client

In [9]:
context = zmq.Context()
sub = context.socket(zmq.SUB)
sub.setsockopt(zmq.SUBSCRIBE, b"")

# keep only the most recent message
sub.setsockopt(zmq.CONFLATE, 1)

# address must match publisher in Processor actor
sub.connect("tcp://127.0.0.1:5555")

<SocketContext(connect='tcp://127.0.0.1:5555')>

In [10]:
def get_buffer():
    """
    Gets the buffer from the publisher
    """
    try:
        b = sub.recv(zmq.NOBLOCK)
    except zmq.Again:
        pass
    else:
        return b
    
    return None

In [11]:
plot = fpl.Plot()

# initialize line plot with pre-allocated with cosine
xs = np.linspace(-10, 10, 100)
ys = np.cos(xs)

plot.add_line(data=np.column_stack((xs, ys)), name="line")

def update_frame(p):
    # recieve memory with buffer
    buff = get_buffer()
    
    if buff is not None:
        # numpy array from buffer
        a = np.frombuffer(buff, dtype=np.float64)
        ix = a[-1]
        # set graphic data
        p["line"].data = a[:-1].reshape(100, 2)
        p.set_title(f"frame: {ix}")
        # change graphic color
        if ix % 2 == 0:
            p["line"].colors = "blue"
        else:
            p["line"].colors = "green"

plot.add_animations(update_frame)
              
plot.show()

RFBOutputContext()

JupyterOutputContext(children=(JupyterWgpuCanvas(), IpywidgetToolBar(children=(Button(icon='expand-arrows-alt'…

In [ ]:
plot.canvas.get_stats()